In [1]:
from project.models.char_baseline import CharSeqBaseline
from project.models.code2vec_solo import Code2VecSolo
from project.models.base_model import _run_model


In [2]:
load_dir = {"logdir":"logs/second_experiment_70dim/second_experiment__var_funcname_otherargs_0808_060151", "mode":"RETURN"}
LOADED = _run_model(CharSeqBaseline, **load_dir)


LOADING FROM: logs/second_experiment_70dim/second_experiment__var_funcname_otherargs_0808_060151, overwriting kwargs


Loading GloVe weights and word to index lookup table


400000it [00:12, 33226.19it/s]


Creating char to index look up table
Tokenizing the word descriptions and characters
Tokenizing the src code
Extracting tensors train and test
INFO:tensorflow:Restoring parameters from logs/second_experiment_70dim/second_experiment__var_funcname_otherargs_0808_060151/BasicModel.ckpt-18818


0811_14:22 WARNING - Loaded from logs/second_experiment_70dim/second_experiment__var_funcname_otherargs_0808_060151: Global Step 18818


In [3]:
sess, nn, data_tuple, step = LOADED

In [6]:
RESULTS = nn.evaluate_bleu(sess, data_tuple.test, max_translations=10000)

In [ ]:
import random
import numpy as np
def prep_samples(data):
    indices = list(range(len(data)))
    sample_size = 400
    no_samples = 1000
    for n in range(no_samples):
        sample_data = [[] for d in data]
        for i in random.choices(indices, k=sample_size):
            for j,d in enumerate(data):
                sample_data[j].append(d[i])
        yield [np.stack(s) for s in sample_data[:-1]] + [sample_data[-1]]
    
bleu_bootstrap = []
for s in prep_samples(data_tuple.test):
    b = nn.evaluate_bleu(sess, s, max_translations=0)
    bleu_bootstrap.append(b[0][0])
print("{:.5f}".format(np.mean(bleu_bootstrap), np.std(bleu_bootstrap)))
    
    

In [34]:
print(len(data_tuple.test[-1]))

4485
